In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from skimage.feature import local_binary_pattern, hog, graycomatrix, graycoprops
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
import pickle
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [2]:
def isskin(image):
    h_min = 0
    h_max = 128
    s_min = 100
    s_max = 150
    v_min = 0
    v_max = 128

    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower_hsv = np.array([h_min, s_min, v_min])
    upper_hsv = np.array([h_max, s_max, v_max])

    skinMask = cv2.inRange(hsv_image, lower_hsv, upper_hsv)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    skinMask = cv2.erode(skinMask, kernel, iterations=2)
    skinMask = cv2.dilate(skinMask, kernel, iterations=2)

    skinMask = cv2.GaussianBlur(skinMask, (5, 5), 0)

    skin = cv2.bitwise_and(image, image, mask=skinMask)

    # alpha = np.uint8(skinMask > 0) * 255
    # result = cv2.merge((skin, alpha))

    return skin

In [3]:
def apply_gamma_correction(image, gamma=1.0):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(image, table)

In [18]:
# Skin Segmentation
def extract_skin_and_preprocess(img):
    img1 = cv2.imread(img)
    skin_img = isskin(img1)

    if skin_img.shape[2] == 4:
        skin_img = skin_img[:, :, :3]
        
    corrected = apply_gamma_correction(skin_img)
    denoised = cv2.fastNlMeansDenoising(corrected, h=10)
    # denoised = cv2.fastNlMeansDenoising(skin_img, h=10)
    img_resized = cv2.resize(denoised, (128, 128))

    # Convert to grayscale
    # gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
    return img_resized

In [19]:
def load_data(dataset_path):
    classes = {'Acne': 0, 'Bags': 1, 'Redness': 2}
    X, y = [], []

    for class_name, label in classes.items():
        class_path = os.path.join(dataset_path, class_name)
        if not os.path.exists(class_path):
            print(f"Folder not found: {class_path}")
            continue
        for subfolder in os.listdir(class_path):
            subfolder_path = os.path.join(class_path, subfolder)
            if os.path.isdir(subfolder_path):
                for img_name in os.listdir(subfolder_path):
                    img_path = os.path.join(subfolder_path, img_name)
                    img = cv2.imread(img_path)
                    if img is not None:
                        img = extract_skin_and_preprocess(img_path)
                        X.append(img)
                        y.append(label)
                    else:
                        print(f"Failed to load image: {img_path}")

    print(f"Loaded {len(X)} images from {len(classes)} classes.")
    return np.array(X), np.array(y)

In [20]:
# Load data
dataset_path = 'Dataset'
X, y = load_data(dataset_path)

Loaded 471 images from 3 classes.


In [27]:
# LBP Feature Extraction
def extract_lbp_features(img, P=8, R=1):
    if len(img.shape) == 3:  # If the image has 3 channels
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img  # Already grayscale
    lbp = local_binary_pattern(gray, P=P, R=R, method='uniform')
    hist, _ = np.histogram(lbp, bins=np.arange(0, P+3), range=(0, P+2))
    hist = hist.astype('float')
    hist /= (hist.sum() + 1e-6)
    return hist

# HOG Feature Extraction
def extract_hog_features(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
    if len(img.shape) == 3:  # If the image has 3 channels
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img  # Already grayscale
    features, _ = hog(gray, orientations=orientations, pixels_per_cell=pixels_per_cell, cells_per_block=cells_per_block, block_norm='L2-Hys', visualize=True)
    return features

# Color Histogram Feature Extraction
def extract_color_features(img, h_range=(0, 128), s_range=(100, 150), v_range=(0, 128)):
    if len(img.shape) != 3 or img.shape[2] < 3:  # Ensure the image has 3 channels
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hist_h = cv2.calcHist([hsv], [0], None, [256], h_range)
    hist_s = cv2.calcHist([hsv], [1], None, [256], s_range)
    hist_v = cv2.calcHist([hsv], [2], None, [256], v_range)
    hist_h /= hist_h.sum() + 1e-6
    hist_s /= hist_s.sum() + 1e-6
    hist_v /= hist_v.sum() + 1e-6
    return np.concatenate([hist_h.flatten(), hist_s.flatten(), hist_v.flatten()])

# GLCM Feature Extraction
def extract_glcm_features(img, distances=[5], angles=[0], properties=('contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation')):
    if len(img.shape) == 3:  # If the image has 3 channels
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img  # Already grayscale
    glcm = graycomatrix(gray, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)
    glcm_features = []
    for prop in properties:
        glcm_features.append(graycoprops(glcm, prop).flatten())
    return np.concatenate(glcm_features)

# Combined Feature Extraction
def extract_combined_features(img, h_range=(0, 128), s_range=(100, 150), v_range=(0, 128)):
    lbp_features = extract_lbp_features(img)
    hog_features = extract_hog_features(img)
    color_features = extract_color_features(img, 
                                            h_range, s_range, v_range)
    # glcm_features = extract_glcm_features(img)
    return np.concatenate([lbp_features, hog_features, color_features])


In [28]:
X_features = np.array([extract_combined_features(img, h_range=(0, 128), s_range=(100, 150), v_range=(0, 128)) for img in X])
X_train, X_val, y_train, y_val = train_test_split(X_features, y, test_size=0.2, random_state=42)

In [29]:
# Train base models
base_models = [
    ('svm', SVC(kernel='linear', probability=True)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42))
]
# Meta-model
meta_model_lr = LogisticRegression(multi_class='multinomial', solver='lbfgs')
meta_model_svm = SVC(kernel='linear', probability=True)
meta_model_rf = RandomForestClassifier(n_estimators=50, random_state=42)

# Stacking Classifier
stacking_lr = StackingClassifier(estimators=base_models, final_estimator=meta_model_lr, cv=5)
stacking_svm = StackingClassifier(estimators=base_models, final_estimator=meta_model_svm, cv=5)
stacking_rf = StackingClassifier(estimators=base_models, final_estimator=meta_model_rf, cv=5)
stacking_lr.fit(X_train, y_train)
stacking_svm.fit(X_train, y_train)
stacking_rf.fit(X_train, y_train)

# Evaluate model
y_pred_lr = stacking_lr.predict(X_val)
y_pred_svm = stacking_svm.predict(X_val)
y_pred_rf = stacking_rf.predict(X_val)

c:\Users\Hansel\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:25:18] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Hansel\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:26:00] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\Hansel\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:26:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } 

In [30]:
print(f"Validation Accuracy Stacking With Logistic Regression With Gamma Correction:")
print(classification_report(y_val, y_pred_lr))
print(f"Validation Accuracy Stacking With Support Vector Machine With Gamma Correction:")
print(classification_report(y_val, y_pred_svm))
print(f"Validation Accuracy Stacking With Random Forest With Gamma Correction:")
print(classification_report(y_val, y_pred_rf))

Validation Accuracy Stacking With Logistic Regression With Gamma Correction:
              precision    recall  f1-score   support

           0       0.47      0.26      0.33        31
           1       0.65      0.85      0.74        33
           2       0.60      0.68      0.64        31

    accuracy                           0.60        95
   macro avg       0.57      0.59      0.57        95
weighted avg       0.58      0.60      0.57        95

Validation Accuracy Stacking With Support Vector Machine With Gamma Correction:
              precision    recall  f1-score   support

           0       0.48      0.32      0.38        31
           1       0.65      0.79      0.71        33
           2       0.62      0.68      0.65        31

    accuracy                           0.60        95
   macro avg       0.58      0.60      0.58        95
weighted avg       0.58      0.60      0.58        95

Validation Accuracy Stacking With Random Forest With Gamma Correction:
          

In [12]:
def find_best_hsv_ranges(X, y, ranges):
    best_accuracy = 0
    best_ranges = None
    for h_range, s_range, v_range in itertools.product(ranges, ranges, ranges):
        print(f"Testing ranges H={h_range}, S={s_range}, V={v_range}")
        X_features = np.array([extract_combined_features(img, h_range, s_range, v_range) for img in X])
        X_train, X_val, y_train, y_val = train_test_split(X_features, y, test_size=0.2, random_state=42)
        pickle.load(open("kent.pkl", "rb"))
        y_pred = model.predict(X_val)
        accuracy = accuracy_score(y_val, y_pred)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_ranges = (h_range, s_range, v_range)
            print(f"New best accuracy: {accuracy} with ranges H={h_range}, S={s_range}, V={v_range}")
    return best_ranges, best_accuracy

# Example usage
ranges = [(0, 128), (128, 256), (50, 200), (100, 255), (100, 150), (50, 150), (80, 175), (80, 200)]
best_ranges, best_accuracy = find_best_hsv_ranges(X, y, ranges)
print("Best Ranges:", best_ranges, "Best Accuracy:", best_accuracy)

NameError: name 'X' is not defined

In [ ]:
def find_best_hsv_ranges(X, y, ranges):
    best_accuracy = 0
    best_ranges = None
    for h_range, s_range, v_range in itertools.product(ranges, ranges, ranges):
        print(f"Testing ranges H={h_range}, S={s_range}, V={v_range}")
        X_features = np.array([extract_combined_features(img, h_range, s_range, v_range) for img in X])
        X_train, X_val, y_train, y_val = train_test_split(X_features, y, test_size=0.2, random_state=42)
        pickle.load(open("kent.pkl", "rb"))
        y_pred = model.predict(X_val)
        accuracy = accuracy_score(y_val, y_pred)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_ranges = (h_range, s_range, v_range)
            print(f"New best accuracy: {accuracy} with ranges H={h_range}, S={s_range}, V={v_range}")
    return best_ranges, best_accuracy

# Example usage
ranges = [(0, 128), (128, 256), (100, 150), (80, 175)]
best_ranges, best_accuracy = find_best_hsv_ranges(X, y, ranges)
print("Best Ranges:", best_ranges, "Best Accuracy:", best_accuracy)